# Attrs

use attr for more eligant and time saving OOP programming

[introduction 1](https://blog.csdn.net/HHG20171226/article/details/103038077)

[github](https://github.com/python-attrs/attrs)
[official document](attrs.org/en/stable/)

1. time saving oop
2. data conversation and validation (vs pydantic)
3. slots feature (vs dataclass)

In [1]:
import attr
import os
from os.path import join as PJ

In [2]:
# 1 auto-create magic methods, get rid of `self`



class Color:
    """
    Color Object for RGB
    """

    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b

    def __repr__(self):
        return f"{self.__class__.__name__}(r={self.r}, g={self.g}, b={self.b})"

    def __lt__(self):
        pass

    def __eq__(self):
        pass


# automatic generate __init__, __repr__, __eq__,
# __ne__, __lt__, __le__, __gt__, __ge__, __hash__

# __ge__ greater equal
# __gt__ geater than
# basically all the dunder method

# 自動處理所有魔術方法

In [3]:

@attr.s()
class GreaterColor:
    r = attr.ib(default=255, type=int)
    g = attr.ib(default=255, type=int)
    b = attr.ib(default=255, type=int)




c1 = Color(255, 255, 255)
print(c1)
c2 = GreaterColor()
print(c2)  # automatic generate __repr__
print(attr.asdict(c2))  # __dict__
print(c2 == GreaterColor(255, 255, 255))  # __eq__
print(c2 > GreaterColor(255, 254, 0))  # __le__

print(c2.__class__.__name__, dir(c2))
print()
print(c1.__class__.__name__, dir(c1))


Color(r=255, g=255, b=255)
GreaterColor(r=255, g=255, b=255)
{'r': 255, 'g': 255, 'b': 255}
True
True
GreaterColor ['__attrs_attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'b', 'g', 'r']

Color ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'b', 'g', 'r']


In [4]:

# attr.s (s for self)
# attr.ib (ib for attribute ? )

# auto_attribs : lay off attr.ib(...)
# forzen : make the instance immutable when it initilized
# kw_only :  set variable of class using keyword only
# slots = True : use __slot__ for attributes(it's will be faster when you are creating short-lifetime object)




In [5]:
# 2 lay off attr-ib


@attr.s
class GreaterColor(object):
    r = attr.ib(default=255, type=int)
    g = attr.ib(default=255, type=int)
    b = attr.ib(default=255, type=int)


# 還要寫 attr.ib 很麻煩
@attr.s(auto_attribs=True)
class EvenGreaterColor:
    r: int = 255
    g: int = 255
    b: int = 255


c2 = GreaterColor()
c3 = EvenGreaterColor()

for c in [c2, c3]:
    print()
    print(c)
    print()
    print(dir(c))



GreaterColor(r=255, g=255, b=255)

['__attrs_attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'b', 'g', 'r']

EvenGreaterColor(r=255, g=255, b=255)

['__annotations__', '__attrs_attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'b', 'g', 'r']


In [6]:
# 3 validation (plug-in based, different with pydnatic)

from attr import attrib


@attr.s
class C(object):
    x = attr.ib()

    @x.validator
    def check(self, attribute, value):
        if value > 42:
            raise ValueError("x must be smaller or equal to 42")


print(C(42))

try:
    print(C(43))
except Exception as e:
    print(e)


def x_smaller_than_y(instance, attribute, value):
    if value >= instance.y:
        raise ValueError("'x' has to be smaller than 'y'!")


@attr.s
class C(object):
    x = attr.ib(validator=[attr.validators.instance_of(int), x_smaller_than_y])
    y = attr.ib()


print(C(x=3, y=4))
try:
    print(C(x=4, y=3))
except Exception as e:
    print(e)


def is_valid_gender(isinstance, attribute,given_gender: str):
    '''
    first 2 arguments is for self and attr
    '''
    if given_gender not in ["male", "female"]:
        raise ValueError(f"gender should be [male, female], got {given_gender}")


@attr.s(auto_attribs=True)
class Person:
    name: str
    gender: str = attrib(validator=is_valid_gender)


try:
    Person("Joe", "BadBoy")
except Exception as e:
    print(e)

C(x=42)
x must be smaller or equal to 42
C(x=3, y=4)
'x' has to be smaller than 'y'!
gender should be [male, female], got BadBoy


In [7]:
# 4 - immutable and converation
@attr.s(frozen=True)
class DataUri:
    basename: str = attr.ib(converter=str) # 將輸入轉換成 string
    prefix: str = attr.ib(converter=str) # 並且是不可變的
    dummy = 1

    @property
    def fullpath(self):
        return PJ(self.prefix, self.basename)

d1 = DataUri(basename="abc.py", prefix=1)
print(d1, d1.fullpath,sep='\n')


try:
    d1.basename = 4  # raise frozen error
except Exception as e:
    print(type(e), e)

DataUri(basename='abc.py', prefix='1')
1/abc.py
<class 'attr.exceptions.FrozenInstanceError'> 


In [8]:
# 5 slot 
# slots is an advance feature in dataclass, only python 3.10+....
# attr has it.

from typing import Optional

immutable = attr.define(auto_attribs=True, frozen=True, kw_only=True, slots=True)


@immutable
class Spreadsheet:
    sheet_id: str
    gid: int = 0
    schema: Optional[dict] = {}
    share_url: str = attr.ib(init=False)
    download_url: str = attr.ib(init=False)

    def __attrs_post_init__(self):
        # https://www.attrs.org/en/stable/init.html
        # setattr, then frozen
        object.__setattr__(
            self, "share_url", f"https://abc/{self.sheet_id}/edf/{self.gid}"
        )
        object.__setattr__(
            self, "download_url", f"https://zxc/{self.sheet_id}/pdq/{self.gid}"
        )
        # self.share_url = f"https://abc/{self.sheet_id}/edf/{self.gid}"

    # @property
    # def share_url(self)
    #     return f"https://abc/{self.sheet_id}/edf/{self.gid}"


s1 = Spreadsheet(sheet_id=1, schema={"poi_name": str})

print(s1)

Spreadsheet(sheet_id=1, gid=0, schema={'poi_name': <class 'str'>}, share_url='https://abc/1/edf/0', download_url='https://zxc/1/pdq/0')
